In [2]:
from keras.layers import Input, Dense, Flatten, Reshape, Conv2D, UpSampling2D, MaxPooling2D, Dropout,BatchNormalization
from keras.models import Model, Sequential
from keras.datasets import mnist
from keras.callbacks import Callback

import numpy as np
import wandb
from wandb.keras import WandbCallback
from keras.regularizers import l1

def add_noise(x_train, x_test):
    # Add some random noise to an image

    noise_factor = 0.4
    x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
    x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape) 
    
    x_train_noisy = np.clip(x_train_noisy, 0., 1.)
    x_test_noisy = np.clip(x_test_noisy, 0., 1.)
    return x_train_noisy, x_test_noisy

run = wandb.init()
config = run.config

config.epochs = 10

(x_train, _), (x_test, _) = mnist.load_data()
(x_train_noisy, x_test_noisy) = add_noise(x_train, x_test)

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

#model = Sequential()
#model.add(Reshape((28,28,1), input_shape=(28,28)))
#model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
#model.add(MaxPooling2D(2,2))
#model.add(BatchNormalization())
#model.add(Dropout(0.2))
#model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
#model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
#model.add(UpSampling2D((2, 2)))
#model.add(Conv2D(1, (3, 3), activation='sigmoid', padding='same'))
#model.add(Reshape((28,28)))


model = Sequential()
model.add(Reshape((28,28,1), input_shape=(28,28)))
model.add(Conv2D(62, (3, 3), activation='relu', padding='same',kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01)))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
#model.add(MaxPooling2D(2,2))
#model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(1, (3, 3), activation='sigmoid', padding='same'))
model.add(BatchNormalization())
model.add(Reshape((28,28)))

model.compile(optimizer='adadelta', loss='mse')

model.summary()

# For visualization
class Images(Callback):
    def __init__(self, validation_data):
        self.validation_data = validation_data
    def on_epoch_end(self, epoch, logs):
        indices = np.random.randint(self.validation_data[0].shape[0], size=8)
        test_data = self.validation_data[0][indices]
        pred_data = self.model.predict(test_data)
        wandb.log({
                  "examples": [
                        wandb.Image(np.hstack([data, pred_data[i]]), caption=str(i))
                        for i, data in enumerate(test_data)]},
                  step=epoch)

#model.fit(x_train, x_train,
#                epochs=config.epochs,
#                validation_data=(x_test, x_test),
#          callbacks=[Images((x_test, x_test)), WandbCallback()])
model.fit(x_train_noisy, x_train,
                epochs=config.epochs,
                validation_data=(x_test_noisy, x_test),
          callbacks=[Images((x_test_noisy, x_test)), WandbCallback()])

model.save('auto-cnn.h5')

epoch,9
val_loss,0.03293
loss,0.03951
_runtime,31
_timestamp,1613937646
_step,9
best_val_loss,0.03293
best_epoch,9


epoch,▁▂▃▃▄▅▆▆▇█
val_loss,█▅▄▃▃▂▂▁▁▁
loss,█▄▃▂▂▂▂▁▁▁
_runtime,▁▂▂▃▄▅▅▆▇█
_timestamp,▁▂▂▃▄▅▅▆▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: wandb version 0.10.19 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 62)        620       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 62)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 62)        248       
_________________________________________________________________
dropout_2 (Dropout)          (None, 14, 14, 62)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        17888     
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 32)      